In [ ]:
# Bonus
# Implementing ResNet-34
import os
import scipy.io as sio
import torch
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Loading labels using file proived on website
labels = sio.loadmat('imagelabels.mat')['labels'][0]
all_image_ids = [f'image_{i+1:05d}.jpg' for i in range(len(labels))]
all_labels = [labels[i] - 1 for i in range(len(labels))]

# Splitting into training, validation, and test sets (80-10-10)
train_id, temp_id, train_labels, temp_labels = train_test_split(
    all_image_ids, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)
val_ids, test_ids, val_labels, test_labels = train_test_split(
    temp_id, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

# Defining custom Dataset class
class FlowerDataset(Dataset):
    def __init__(self, image_ids, labels, img_dir, transform=None):
        self.image_ids = image_ids
        self.labels = labels
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.image_ids[idx])
        image = Image.open(img_name).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Paths to my drive folder
img_dir = '/content/drive/MyDrive/jpg'

# Creating datasets using dataset class defined above
train_dataset = FlowerDataset(train_id, train_labels, img_dir, transform=transform)
val_dataset = FlowerDataset(val_ids, val_labels, img_dir, transform=transform)
test_dataset = FlowerDataset(test_ids, test_labels, img_dir, transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# `train_loader`, `val_loader`, and `test_loader` are ready for training, validation, and testing


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau


# data augmentation for training
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Defining ResNet-34 architecture
class SimpleBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(SimpleBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = nn.ReLU()(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = nn.ReLU()(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=102):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(512 * block.expansion, num_classes)
        )

    def _make_layer(self, block, planes, num_blocks, stride):
        layers = [block(self.in_planes, planes, stride)]
        self.in_planes = planes * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Instantiating the model, optimizer and scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet(SimpleBlock, [3, 4, 6, 3]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


In [ ]:
# Traininging function with accuracy and loss tracking
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=30):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        # Training Phase
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (preds == labels).sum().item()

        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = correct_train / total_train
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_accuracy)

        # Validation Phase
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (preds == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct_val / total_val
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)

        # Scheduler step
        scheduler.step(avg_val_loss)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, "
              f"Train Acc: {train_accuracy * 100:.2f}%, Val Acc: {val_accuracy * 100:.2f}%, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

    return train_losses, val_losses, train_accuracies, val_accuracies

# training
train_losses, val_losses, train_accuracies, val_accuracies = train_and_validate(
    model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=30
)

Epoch [1/30], Train Loss: 4.2524, Val Loss: 3.6510, Train Acc: 6.46%, Val Acc: 10.13%, LR: 0.001000


Epoch [2/30], Train Loss: 3.6453, Val Loss: 3.3695, Train Acc: 11.20%, Val Acc: 14.41%, LR: 0.001000


Epoch [3/30], Train Loss: 3.3869, Val Loss: 3.2767, Train Acc: 14.41%, Val Acc: 15.75%, LR: 0.001000


Epoch [4/30], Train Loss: 3.1928, Val Loss: 3.1472, Train Acc: 17.55%, Val Acc: 22.34%, LR: 0.001000


Epoch [5/30], Train Loss: 2.9984, Val Loss: 2.8333, Train Acc: 22.07%, Val Acc: 25.64%, LR: 0.001000


...

Epoch [26/30], Train Loss: 0.1878, Val Loss: 1.2034, Train Acc: 94.38%, Val Acc: 71.79%, LR: 0.000500


Epoch [27/30], Train Loss: 0.1011, Val Loss: 1.2615, Train Acc: 97.50%, Val Acc: 72.04%, LR: 0.000500


Epoch [28/30], Train Loss: 0.0833, Val Loss: 1.1913, Train Acc: 98.09%, Val Acc: 72.16%, LR: 0.000500


Epoch [29/30], Train Loss: 0.0817, Val Loss: 1.2188, Train Acc: 98.02%, Val Acc: 73.26%, LR: 0.000500


Epoch [30/30], Train Loss: 0.0637, Val Loss: 1.2793, Train Acc: 98.40%, Val Acc: 73.26%, LR: 0.000500


# References

### PyTorch
- [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)
- [torchvision.transforms](https://pytorch.org/vision/stable/transforms.html)
- [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)
- [DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) 
- [nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
- [nn.BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)
- [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
- [torch.optim](https://pytorch.org/docs/stable/optim.html)
- [ReduceLROnPlateau](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau)

### ResNet Architecture
- [ResNet Paper (arXiv)](https://arxiv.org/abs/1512.03385)
- [torchvision.models.resnet](https://pytorch.org/vision/stable/models.html#id2)
